<a href="https://colab.research.google.com/github/Flychuban/Space-Soup/blob/main/TextSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.8 MB/s eta 0:00:00


In [2]:
import requests
from transformers import pipeline
from bs4 import BeautifulSoup
import re

In [3]:
# Load summarization model from HuggingFace
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Get text from web page

In [4]:
from datetime import datetime
def search_urls_space_news():
  day = int(datetime.now().day)
  month= int(datetime.now().month)
  year = int(datetime.now().year)

  URL = f"https://www.google.com/search?q=space.com+latest+news&tbm=nws"
  print(URL)
  r = requests.get(URL)
  soup = BeautifulSoup(r.text, 'html.parser')
  atags = soup.find_all(['a'])
  hrefs = [link['href'] for link in atags]
  return hrefs


In [5]:
hrefs = search_urls_space_news()

https://www.google.com/search?q=space.com+latest+news&tbm=nws


In [6]:
print(hrefs)

['/?sa=X&ved=0ahUKEwju8qbilNOBAxWTm2oFHZIyBcAQOwgC', '/search?q=space.com+latest+news&tbm=nws&sca_esv=569753216&ie=UTF-8&gbv=1&sei=e4MYZa7JOJO3qtsPkuWUgAw', '/search?q=space.com+latest+news&sca_esv=569753216&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwju8qbilNOBAxWTm2oFHZIyBcAQ_AUIBSgA', '/search?q=space.com+latest+news&sca_esv=569753216&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwju8qbilNOBAxWTm2oFHZIyBcAQ_AUIBygC', '/search?q=space.com+latest+news&sca_esv=569753216&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwju8qbilNOBAxWTm2oFHZIyBcAQ_AUICCgD', 'https://maps.google.com/maps?q=space.com+latest+news&um=1&ie=UTF-8&sa=X&ved=0ahUKEwju8qbilNOBAxWTm2oFHZIyBcAQ_AUICSgE', '/search?q=space.com+latest+news&sca_esv=569753216&ie=UTF-8&tbm=shop&source=lnms&sa=X&ved=0ahUKEwju8qbilNOBAxWTm2oFHZIyBcAQ_AUICigF', '/search?q=space.com+latest+news&sca_esv=569753216&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwju8qbilNOBAxWTm2oFHZIyBcAQ_AUICygG', '/advanced_search', '/search?q=space.com+latest+news&sca_esv=56

In [7]:
exclude_list = ['maps', 'policies', 'preferences', 'accounts', 'support']

In [8]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls:
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [9]:
cleaned_urls = strip_unwanted_urls(hrefs, exclude_list)
cleaned_urls

['https://www.space.com/gravity-affects-matter-antimatter-similarly',
 'https://www.google.com/search?q%3Dspace.com%2Blatest%2Bnews%26tbm%3Dnws%26pccc%3D1',
 'https://www.space.com/historic-donut-black-hole-is-spinning-new-study-finds',
 'https://www.space.com/spacex-starlink-launch-group-6-19',
 'https://www.space.com/osiris-rex-asteroid-sample-caniser-lid-lifted-photos',
 'https://www.space.com/mars-rover-perseverance-dust-devil-august-2023',
 'https://www.space.com/harvest-full-moon-will-illuminate-the-sky-on-september-29',
 'https://www.space.com/spacex-launch-psyche-asteroid-mission-one-week',
 'https://www.space.com/james-webb-space-telescope-evolved-galaxy-early-universe',
 'https://www.space.com/iran-launches-noor-3-imaging-satellite-september-2023',
 'https://www.tweaktown.com/news/93488/scientists-gasped-after-they-opened-the-lid-of-nasas-asteroid-sample-capsule/index.html']

In [10]:
def get_text_from_urls(urls):
  articles = []
  titles = []
  for url in urls:
    headers = {'User-Agent': 'Mozilla/5.0'}
    cookies = {"CONSENT": "YES+cb.20210720-07-p0.en+FX+410"}
    r = requests.get(url, headers=headers, cookies=cookies)
    if r.status_code != 200:
      continue

    soup = BeautifulSoup(r.text, 'html.parser')
    paragraphs = soup.find_all(['p'])
    text = [paragraph.text for paragraph in paragraphs]
    # words = ' '.join(text)
    article = ' '.join(text)
    article = article.rstrip()

    # Minimal number of symbols in article, prevent returning cookies or terms error
    if len(article) < 200:
      continue

    articles.append(article)
    url_h1_tags = soup.find_all(['h1'])
    url_titles = [title.text.strip() for title in url_h1_tags]
    titles.append(url_titles)

  return titles, articles

In [11]:
titles, articles = get_text_from_urls(cleaned_urls)

In [12]:
articles[1]

'When you purchase through links on our site, we may earn an affiliate commission. Here’s how it works. The historic \'donut\' black hole M87* is, in fact, spinning. The supermassive black hole M87*, which rose to fame in 2019 when it became the first void to be imaged and revealed a fuzzy orange donut (then later sharpened by AI into a skinny ring), is now confirmed to be spinning. The announcement of this update came on Wednesday (Sept. 27). However, as to how fast M87* is spinning? That\'s not yet known. For two decades, a network of radio telescopes have been eyeing the black hole, which resides in the heart of the Messier 87 (M87) galaxy about 55 million light-years away from Earth in the constellation Virgo. These instruments have been especially intrigued by a powerful jet of radiation and particles blasting from the black hole\'s poles, and according to new results, that relativistic jet appears to be swinging like a pendulum on a 11-year cycle. Scientists say this is because o

Split text from the article into chunks

In [13]:
sentences = []
for i, article in enumerate(articles):
  article = article.replace('.', '.<eos>')
  article = article.replace('!', '!<eos>')
  article = article.replace('?', '?<eos>')
  sentences.append(article.split('<eos>'))

In [14]:
max_tokens_chunk = 500
chunks = []

for i, article_sentences in enumerate(sentences):
  chunks.append([])
  current_chunk = 0
  for sentence in article_sentences:
    if len(chunks[i]) == current_chunk + 1:
      if len(chunks[i][current_chunk]) + len(sentence.split(' ')) <= max_tokens_chunk:
        chunks[i][current_chunk].extend(sentence.split(' '))
      else:
        current_chunk += 1
        chunks[i].append(sentence.split(' '))
    else:
      print(current_chunk)
      chunks[i].append(sentence.split(' '))

0
0
0
0
0
0
0
0
0
0


In [15]:
joined_chunks = []

for i, article_chunks in enumerate(chunks):
    article_text = []

    for chunk_id in range(len(article_chunks)):
        article_text.append(' '.join(article_chunks[chunk_id]))

    joined_chunks.append(article_text)

In [16]:
joined_chunks[0][1]

' According to the new study, that conclusively demonstrated that gravity causes the antihydrogen to fall downward.  "This gives us a powerful experimental knob that allows us, basically, to believe the experiment actually worked because we can prove to ourselves that we can control the experiment in a predictable manner," Joel Fajans, a physics professor at UC Berkeley and a co-author of the new study, said in the statement.  The team also found that the gravitational acceleration of antihydrogen was close to that of normal matter, which is 9. 8 meters (32 feet) per second squared.  That result is expected to hold true for other antimatter particles too, researchers say. \xa0 "It would be doubly surprising if this was not true (first, that something fell up, and second that there was a difference with antihydrogen)," Fajans told Space. com in an email.  However, though the latest findings rule out theories that posit antimatter is repelled by gravity, only more precise measurements wi

Summarize all chunks

In [17]:
def summarize_chunks(chunks):
  summarized_texts = []
  for article_chunks in chunks:
    print(len(article_chunks))
    article_text = []
    result = summarizer(article_chunks, max_length=120, min_length=30, do_sample=False)
    article_text = ' '.join([summ['summary_text'] for summ in result])
    summarized_texts.append(article_text)
  return summarized_texts

In [18]:
summarized_texts = summarize_chunks(joined_chunks)

3
2
2
2
2
3
2
3


Your max_length is set to 120, but your input_length is only 24. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=12)


2
1


In [19]:
summarized_texts[-1]

" OSIRIS-REx landed asteroid samples for the very first time back on Earth this week . The container containing the samples was immediately transported to NASA's Johnson Space Center in Houston, Texas, and has now been cracked open . NASA will reveal the sample to the world via a livestream scheduled for 11 a.m. ET on October 11 ."

In [40]:
merge_phrases = ["Now, let's shift our focus to: ",
"Moving on to another intriguing development in space exploration:",
"Switching gears, let's delve into: ",
"Turning our attention to a different aspect of space science: ",
"Next up on our cosmic journey: ",
"Shifting our gaze to a separate celestial event: ",
"Now, let's explore another celestial headline: ",
"Transitioning to a different corner of the universe: ",
"Moving forward, let's discuss: ",
"Steering our spacecraft of knowledge toward: ",
"Now, in a different part of the cosmos: ",
"Switching our telescopes to a new cosmic phenomenon: ",
"Changing our trajectory to focus on: ",
"Stepping into another dimension of space news: ",
"Now, as we orbit to the next topic: ",
"Zooming in on a separate celestial occurrence: ",
"Leaving our previous topic behind, let's explore: ",
"Pivoting to a fresh cosmic revelation: ",
"Our journey through the universe continues with: ",
"Now, let's set our sights on: "]

all_summarized_text = ""
for i, summarized_text in enumerate(summarized_texts):
  all_summarized_text = all_summarized_text + "... " + f"{merge_phrases[i%(len(merge_phrases))]}" + summarized_text

In [41]:
all_summarized_text

'... Now, let\'s shift our focus to:  An international team of physicists reported a major finding about an elusive form of matter known as antimatter . It appears that antimatter responds to gravity the same way regular matter does . This result marks the first-ever direct observation of free-falling antimatter, in which atoms are made of antiprotons instead of protons and antielectrons .  Antihydrogen\'s gravitational acceleration is close to that of normal matter, researchers say . The gravitational acceleration of antihydrogen is 9. 8 meters (32 feet) per second squared squared . That result is expected to hold true for other antimatter particles too, UC Berkeley scientists say . Researchers mark the beginning of detailed and direct pursuit of the gravitational nature of antimatter .  The findings were published by the Antihydrogen Laser Physics Apparatus (ALPHA) collaboration at CERN on Wednesday (Sept. 27) in the journal Nature . Most would say it is unlikely, but still worth pur

Create a news article with all of the summarized texts

NLP for news reading

In [25]:
!pip install gTTS pydub

In [42]:
from gtts import gTTS
from pydub import AudioSegment
from pydub.effects import speedup


tts = gTTS(text=all_summarized_text, lang='en')
filename='voice.mp3'
tts.save(filename)
audio = AudioSegment.from_mp3(filename)
new_file = speedup(audio,1.2,150)
new_file.export(filename, format="mp3")

<_io.BufferedRandom name='voice.mp3'>